In [18]:
import pandas as pd
import numpy as np
import random as rd
import sqlite3
import math

In [21]:
# list of postal codes in zone
zone = [75001, 75002, 75003, 75004, 75005, 75006, 75007, 75008, 75009, 75010, 75011, 75012, 75013, 75014, 75015]
nb_of_zips = len(zone)

#DISTANCE MATRIX
dist_matrix = np.zeros([nb_of_zips, nb_of_zips])
dist_matrix[np.triu_indices(nb_of_zips, 1)] = [rd.randint(1,10) for i in range(int((nb_of_zips-1)*nb_of_zips*.5))]
dist_matrix += dist_matrix.T

print(dist_matrix)

# Matrice des disponibilités pour chaque code postal, qu'on peut update de manière asynchrone.
days_planned = 14
available_timeslots = pd.DataFrame(np.array([[0 for zip in zone] for i in range(days_planned)]), columns=zone)


#UPDATE TIMESLOT AVAILABILITY

def get_dist(zip, previous, next, dist_matrix=dist_matrix, zone=zone):
    assert zip in zone, 'zip isnt in zone anyway'

    if math.isnan(previous) or math.isnan(next):
        return 1000000
    else:
        previous, next = int(previous), int(next)

    if not ((previous in zone) and (next in zone)):
        return 1000000

    dist = dist_matrix[zone.index(zip), zone.index(previous)] + dist_matrix[zone.index(zip), zone.index(next)]
    return dist

def timeslot_availabilities(timeslot, zone=zone, dist_limit=10):
    """ 
    EXECUTE SQL QUERY to get previous and next visit adress

    connector = sqlite3.connect()
    query_template = pd.read_sql_query('../query_template/previous_and_next_visit.sql', connector)
    fill sql query template with timeslot
    
    execute sql query
    available_techs = query.result
    """
    available_techs = pd.read_csv('../../available_techs.csv')


    timeslot_availability = []
    for zip in zone:
        for index, row in available_techs.iterrows():
            dist = get_dist(zip, row['previous_visit_postal_code'], row['next_visit_postal_code'])
            isAvailable = (dist <= dist_limit)
            if isAvailable:
                break

        timeslot_availability.append(isAvailable*1)
    return timeslot_availability

timeslots = [0, '...', 'insert timeslot datestamps in this list']

av = timeslot_availabilities(timeslots[0])
print('availability for timeslot...', av)



[[ 0.  6.  4.  9.  9.  2.  8.  1.  5.  9.  6.  3.  7.  4. 10.]
 [ 6.  0. 10.  5.  7.  9.  3.  7.  9.  1.  8.  3.  4.  5.  5.]
 [ 4. 10.  0. 10.  4.  1.  9.  6. 10.  5. 10.  5.  9.  6.  5.]
 [ 9.  5. 10.  0.  8.  8.  1.  2. 10.  6.  2.  8.  4.  1.  4.]
 [ 9.  7.  4.  8.  0.  3.  5.  1. 10.  2.  6.  5.  4.  9. 10.]
 [ 2.  9.  1.  8.  3.  0.  1.  2.  7.  8.  8.  8. 10.  9.  6.]
 [ 8.  3.  9.  1.  5.  1.  0.  9.  3.  4.  9.  5.  6.  1.  6.]
 [ 1.  7.  6.  2.  1.  2.  9.  0.  4.  7.  4. 10.  8.  7.  9.]
 [ 5.  9. 10. 10. 10.  7.  3.  4.  0.  7.  5.  6.  5. 10.  5.]
 [ 9.  1.  5.  6.  2.  8.  4.  7.  7.  0.  1.  7.  7.  2.  6.]
 [ 6.  8. 10.  2.  6.  8.  9.  4.  5.  1.  0.  4. 10.  4.  2.]
 [ 3.  3.  5.  8.  5.  8.  5. 10.  6.  7.  4.  0.  9.  1.  8.]
 [ 7.  4.  9.  4.  4. 10.  6.  8.  5.  7. 10.  9.  0.  9.  4.]
 [ 4.  5.  6.  1.  9.  9.  1.  7. 10.  2.  4.  1.  9.  0.  2.]
 [10.  5.  5.  4. 10.  6.  6.  9.  5.  6.  2.  8.  4.  2.  0.]]
availability for timeslot... [0, 1, 0, 1, 0, 0, 0, 0, 